In [62]:
import sys, os, json, subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [63]:
import sit_helper
standard_import_tool_plugin_path=sit_helper.load_standard_import_tool_plugin()
import cbm3_python_helper
cbm3_python_helper.load_cbm3_python()
import cbm3_python.simulation.projectsimulator
from cbm3_python.cbm3data import cbm3_results
from cbm3_python.cbm3data.aidb import AIDB
from cbm3_python.cbm3data.accessdb import AccessDB
from cbm3_python.cbm3data.projectdb import ProjectDB
from cbm3_python.simulation.simulator import Simulator

In [64]:
local_working_dir = os.path.abspath(".")
toolbox_path = r'C:\Program Files (x86)\Operational-Scale CBM-CFS3'
aidb_path=os.path.join(toolbox_path, "admin", "dbs", "ArchiveIndex_Beta_Install.mdb")
cbm_exe_path=os.path.join(toolbox_path, "admin", "executables")
cbm_wd = os.path.join(toolbox_path, "temp")

In [65]:
def create_event_factory(events):
    def factory(config):
        for e in events:
            config.add_event(**e)
    return factory
            
def create_transition_rule_factory(transitions):
    def factory(config):
        for t in transitions:
            config.add_transition_rule(**t)
    return factory

In [66]:
def iteration_import(project_path, config_save_path, events_factory, transition_factory):


    c = sit_helper.SITConfig(
        imported_project_path=project_path,
        initialize_mapping=True
    )
    c.data_config(
        age_class_size=5,
        num_age_classes=14,
        classifiers=["Stand ID", "Leading Species", "Land Cover"])

    c.set_single_spatial_unit(17)
    c.set_species_classifier("Leading Species")
    c.map_species("Nonforest", "Not stocked")

    c.add_inventory(classifier_set=["Stand 1", "Luvisolic (E. Canada)", "Nonforest"],
                    area=100, age=0, unfccc_land_class=1)
    c.add_inventory(classifier_set=["Stand 2", "Spruce", "Forest Only"],
                    area=1000, age=0, unfccc_land_class=0,
                   historic_disturbance="Wildfire", last_pass_disturbance="Wildfire")

    growth_curve_data = [0,0,0,27,84,155,217,271,317,353,382,406,426,426]

    not_stocked_growth = [0.0]*len(growth_curve_data)

    c.add_yield(classifier_set=["?","?","Nonforest"],
                leading_species_classifier_value="Not stocked",
                values=not_stocked_growth)

    c.add_yield(classifier_set=["?","Red pine","Forest Only"], 
                leading_species_classifier_value="Red pine", values=growth_curve_data)
    
    c.add_yield(classifier_set=["?","Spruce","Forest Only"], 
                leading_species_classifier_value="Spruce", values=growth_curve_data)
    
    events_factory(c)
    transition_factory(c)
    
    c.import_project(standard_import_tool_plugin_path, config_save_path)

    

In [67]:
def get_project_path(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_{}.mdb".format(iteration_num))

def get_results_db_path(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_results_{}.mdb".format(iteration_num))

def get_config_save_path(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_config_{}.mdb".format(iteration_num))

def get_iteration_tempfiles_dir(iteration_num):
    return os.path.join(local_working_dir, "iterative_cbm", "iterative_cbm_tempfiles_{}.mdb".format(iteration_num))


In [68]:
events = [
    {
    "classifier_set": ["?","?","?"],
    "disturbance_type": "Afforestation", 
    "time_step": 1, 
    "target": 100,
    "target_type": "Area",
    "sort": "SORT_BY_SW_AGE"
    }
]
transitions = [
    {
    "classifier_set_source": ["?","?","?"],
    "classifier_set_target": ["?","Red pine","Forest Only"],
    "disturbance_type": "Afforestation",
    "percent": 100
    }
]

iteration_num = 1
project_path = get_project_path(iteration_num)
tempfiles_dir = get_iteration_tempfiles_dir(iteration_num)
results_db_path = get_results_db_path(iteration_num)

iteration_import(
    project_path,
    get_config_save_path(iteration_num),
    create_event_factory(events),
    create_transition_rule_factory(transitions))



In [69]:
with AIDB(aidb_path, False) as aidb, \
     AccessDB(project_path, False) as proj:

    
    simId = aidb.AddProjectToAIDB(proj)
    s = Simulator(cbm_exe_path,
                simId,
                os.path.dirname(project_path),
                cbm_wd,
                toolbox_path)
    

    s.CleanupRunDirectory()
    s.CreateMakelistFiles()
    s.copyMakelist()
    s.runMakelist()
    s.loadMakelistSVLS()
    s.DumpMakelistSVLs()
    s.CreateCBMFiles()
    s.CopyCBMExecutable()
    s.RunCBM()
    s.CopyTempFiles(output_dir=tempfiles_dir)
    s.LoadCBMResults(output_path = results_db_path)